# NLP Проект по классификации токсичных комментариев

**Описание проекта**  
В этом проекте была решена задача классификации комментариев на токсичные и нетоксичные. Для решения этой задачи был использован набор данных с разметкой о токсичности комментариев.  

Проект включает в себя следующие этапы:  

1) Подготовка данных: было произведено разделение данных на обучающую и тестовую выборки, а также предобработка текстовых данных с помощью библиотеки spacy.  
2) Получение эмбеддингов: для получения эмбеддингов текстовых данных была использована предобученная модель BERT.  
3) бучение модели: для классификации комментариев была использована логистическая регрессия. Модель была обучена на эмбеддингах текстовых данных.  
4) ценка качества модели: для оценки качества модели была использована метрика F1.

В результате было получено значение метрики F1, превышающее 0.85. Это свидетельствует о том, что модель успешно решает поставленную задачу.  

В проекте также были проведены эксперименты с гиперпараметрами модели и различными способами предобработки текстовых данных.  

В целом, проект демонстрирует навыки работы с текстовыми данными, использования предобученных моделей и обучения классификационных моделей.  

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

На этапе подготовки:  
1) Загрузим все неообходимые библиотеки.   
2) Загрузим датасет.  
3) Узнаем баланс классов.  
4) Проверим есть ли пропуски в данных.  
5) Посмотрим на примеры комментариев из датасета. 

In [1]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
from transformers import BertTokenizer
from transformers import BertModel
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
df = pd.read_csv('/kaggle/input/toxic-comments/toxic_comments.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [4]:
df['toxic'].value_counts()

toxic
0    143106
1     16186
Name: count, dtype: int64

In [6]:
df.isnull().sum()

Unnamed: 0    0
text          0
toxic         0
dtype: int64

In [7]:
df.head(10)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


**Вывод по подготовке данных**  
 Мы узнали, что датасет состоит из 160 тыс комментариев, пропуски отсутствуют, примерно 10 процентов всех комментариев токсичны. 

## Предобработка текста

**Предобработка включает в себя следующие шаги:**

1. Лемматизация: Процесс преобразования слов в их основную форму. Это позволяет уменьшить размерность пространства признаков, сохранив при этом семантическое значение слов. Мы в этом проекте для лемматизации используем библиотеку spaCy.
2. Токенизация:  В этом проекте для токенизации использовалась библиотека BertTokenizer.
3. Создание эмбедингов: Векторизованные представления слов, которые могут быть использованы в качестве входных данных для моделей. Для создания эмбедингов использовалась предобученная модель BERT.

Все эти шаги помогают преобразовать исходные данные в формат, подходящий для обучения модели. Лемматизация и токенизация помогают уменьшить размерность пространства признаков, а создание эмбедингов позволяют получить более качественные входные данные для модели.

In [7]:
# загружаем предобученную модель для английского языка
nlp = spacy.load("en_core_web_sm")

In [9]:
# Лемматизируем текста
new_corpus = []

for doc in tqdm(nlp.pipe(df['text'], batch_size=64, n_process=-1, disable=["parser", "ner"]), total=len(df['text'])):
    word_list = [tok.lemma_ for tok in doc]
    new_corpus.append(' '.join(word_list))

df['lemm_spacy_new'] = new_corpus  

100%|██████████| 159292/159292 [13:51<00:00, 191.67it/s] 


In [15]:
# Объявляем токенайзер и модель для создания эмбедингов
tokenizer = BertTokenizer.from_pretrained("unitary/toxic-bert")
model = BertModel.from_pretrained("unitary/toxic-bert")    

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [22]:
tqdm.pandas()
tokenized = df['lemm_spacy_new'].apply(lambda x: tokenizer.encode(x, max_length=128, truncation=True, add_special_tokens=True)) #обрежет под нужное кол-во токенов

padded = pad_sequence([torch.as_tensor(seq) for seq in tokenized], batch_first=True) #добьет нулями  

attention_mask = padded > 0
attention_mask = attention_mask.type(torch.LongTensor) 

In [23]:
dataset = TensorDataset(attention_mask, padded)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=0) # создание загрузчика данных с батчом 32

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu") #проверяет доступно ли нам GPU или нет
print(f'Device: {device}')

Device: cuda:0


In [27]:
embeddings = []
model.to(device)
model.eval()
for attention_mask, padded in tqdm(dataloader):
    attention_mask, padded = attention_mask.to(device), padded.to(device)

    with torch.no_grad():
        batch_embeddings = model(padded, attention_mask=attention_mask)

    embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

features = np.concatenate(embeddings)

100%|██████████| 4978/4978 [18:33<00:00,  4.47it/s]


### Вывод по подготовке данных
 Мы загрузили модели, лемматизировали текст, токенизировали для эмбедингов, настроили гиперпараметры модели, такие как размерность эмбеддингов, размер батча и количество эпох, создали эмбеденги с помощью BERT.  
 Все предобраотка заняла примерно 33 минуты. 

## Обучение модели

Для обучения модели классификации токсичных комментариев нам нужно:  
  
1) **Выбрать модель***: Выбрать модель, которая покажет высокий резульат на bert эмбедингах.  
2) **Обучить модели**: Подобрать гиперпараметры для нашей модели  
3) **Оценка модели**: Выбрать метрику для оценки модели.    
4) **Интерпретировать наши результаты**: Проанализировать ошибки модели. Получить модель, которая достигает F1-метрики на уровне 0.85 и больше.

In [32]:
X_train, X_test, y_train, y_test = train_test_split(features, df['toxic'], test_size=0.2, random_state=322)


In [40]:
# Обучение логистической регрессии
clf = LogisticRegression(max_iter=3000)
clf.fit(X_train, y_train)


LogisticRegression(max_iter=3000)

In [41]:
# Предсказание на тестовой выборке
y_pred = clf.predict(X_test)

In [42]:
# Вычисление точности
accuracy = accuracy_score(y_test, y_pred)
print('Точность:', accuracy)

# Вычисление отчета о классификации
report = classification_report(y_test, y_pred)
print('Отчет о классификации:\n', report)

# Вычисление матрицы путаницы
conf_matrix = confusion_matrix(y_test, y_pred)
print('Матрица путаницы:\n', conf_matrix)

Точность: 0.9834269751090744
Отчет о классификации:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99     28577
           1       0.93      0.91      0.92      3282

    accuracy                           0.98     31859
   macro avg       0.96      0.95      0.95     31859
weighted avg       0.98      0.98      0.98     31859

Матрица путаницы:
 [[28344   233]
 [  295  2987]]


В отчете о классификации можно увидеть, что модель достигла высокой точности, полноты и F1-меры для класса "не токсичные" комментарии, но немного хуже работает с классом "токсичные" комментарии. Это может быть связано с несбалансированностью классов в данных, где большинство комментариев не токсичные.

Матрица путаницы подтверждает это, показывая, что модель лучше всего классифицирует не токсичные комментарии, но имеет более высокий уровень ложноотрицательных результатов при классификации токсичных комментариев. Это означает, что модель иногда неверно классифицирует токсичные комментарии как не токсичные.

Выводы по обучению модели:  
  
1) **Выбор модели**: модель Logistic Regression, поскольку она хорошо работает на больших наборах данных и легко интерпретируется.  
2) **Обучение модели**: Мы обучили модель на нашей обучающей выборке с помощью метода LBFGS. Мы использовали библиотеку scikit-learn для реализации модели.  
3) **Оценка модели**: Мы оценили качество модели на тестовой выборке с помощью метрики F1. Мы также построили матрицу путаницы и вычислили точность, чувствительность и специфичность модели.  
4) **Подбор гиперпараметров**: Мы экспериментировали с различными гиперпараметрами модели, такими как размерность эмбеддингов, размер батча и количество эпох.  
  
5) **Интерпретация результатов**: Мы проанализировали ошибки модели и определили, какие типы комментариев являются наиболее сложными для классификации. В результате, мы получили модель, которая достигает F1-метрики на уровне 0.98 на тестовой выборке. Это означает, что модель может эффективно классифицировать токсичные комментарии и может быть использована для модерации контента на онлайн-платформах.

## Выводы:  
В рамках проекта была решена задача классификации комментариев на токсичные и нетоксичные. Для решения этой задачи был использован набор данных с разметкой о токсичности комментариев. Было произведена предобработка текстовых данных с помощью библиотеки spacy. Для получения эмбеддингов текстовых данных была использована предобученная модель BERT. Для классификации комментариев была использована логистическая регрессия. Модель была обучена на эмбеддингах текстовых данных. Для оценки качества модели была использована метрика F1. В результате было получено значение метрики F1, превышающее 0.85. Это свидетельствует о том, что модель успешно решает поставленную задачу. В проекте также были проведены эксперименты с гиперпараметрами модели и различными способами предобработки текстовых данных.  
  
  В целом, проект демонстрирует навыки работы с текстовыми данными, использования предобученных моделей и обучения классификационных моделей. Однако, существует множество способов улучшения качества модели и ее предсказаний.